In [1]:
from bs4 import BeautifulSoup
import warnings
import requests
from datetime import datetime
# import pandas as pd
warnings.filterwarnings("ignore")

In [8]:
# main_url ="https://edwardzou.blogspot.com/?fbclid=IwAR3HsqRaK0D4sbuDJX7agJwfQc2uLBtc9V0PRJR0uT3u3pGEF0F9AL3JVVw"
# url = main_url
# response = requests.get(url)
# html = BeautifulSoup(response.text)
# boxes = html.find("div", id="content-wrapper").find_all("div",class_="box-body-deg")
for box in boxes:
    print(box.find("a")["title"])
    print(box.find("a")["href"])


In [7]:
# def crawler_main(pages, newest_update):
pages = 1
newest_update = "2019-06-15 00:00"
main_url = "https://nba.udn.com/nba/index?gr=www"
domain_url = "https://nba.udn.com"
# go to main page > 焦點新聞 more
newest_update = datetime.strptime(newest_update, "%Y-%m-%d %H:%M")
url = main_url
response = requests.get(url)
html = BeautifulSoup(response.text)
news_page = domain_url + html.find("div", id="mainbar").find("div", id="news").find("h1", class_="box-title").find("a")["href"]
# https://nba.udn.com/nba/cate/6754
# result at 0617
# news_page = "https://nba.udn.com/nba/cate/6754"
# go page1_newest
news_articles = []

In [11]:
# def crawler_main(pages, newest_update):
pages = 1
newest_update = "2019-06-15 00:00"
main_url = "https://nba.udn.com/nba/index?gr=www"
domain_url = "https://nba.udn.com"
# go to main page > 焦點新聞 more
newest_update = datetime.strptime(newest_update, "%Y-%m-%d %H:%M")
url = main_url
response = requests.get(url)
html = BeautifulSoup(response.text)
news_page = domain_url + html.find("div", id="mainbar").find("div", id="news").find("h1", class_="box-title").find("a")["href"]
# https://nba.udn.com/nba/cate/6754
# result at 0617
# news_page = "https://nba.udn.com/nba/cate/6754"
# go page1_newest
news_articles = []
for c_page in range(1, pages + 1):
    url = news_page + "/-1/newest/" + str(c_page)
    response = requests.get(url)
    html = BeautifulSoup(response.text)
    body_box = html.find("div", id="news_list_body").find("dl").find_all("dt")
    b = 0
    for box in body_box:
        url = domain_url + box.find("a")["href"]
        update_time = get_article_update_time(url)
        check_update_time = datetime.strptime(update_time, "%Y-%m-%d %H:%M")
        title = box.find("h3").text
        if check_update_time <= newest_update:
            b = 1
            break
        content = get_article_content(url)
        news_articles.append([url, title, update_time, content])
    if b:
        break
news_articles

[['https://nba.udn.com/nba/story/6780/3881113',
  '米道頓不執行球員選項 有望續約公鹿',
  '2019-06-19 17:10',
  ' Getty Images facebooktwitterpinterest27歲的米道頓（Khris Middleton）可以說是今年熱門自由球員之一，不少球隊都有意爭取，不過根據近期消息，米道頓今年夏天將會跳出合約，並且將與公鹿完成續約。米道頓堪稱是這幾年來的二輪新秀之光其中一員，他在2012年選秀會上以第二輪39順位加入活塞，隔年就在詹寧斯（Brandon Jennings）和奈特（Brandon Knight）的交易案中被當作籌碼送到密爾瓦基，這筆交易也讓米道頓的NBA生涯產生巨大轉折。加入公鹿，米道頓個人表現逐漸提升，上季平均攻下18.3分4.3助攻6籃板，每場投進2.3顆三分球，助攻、籃板、三分球分別創下生涯新高，並幫助公鹿一路殺進東區決賽。米道頓攻守兩端的表現也讓他成為今夏熱門球員之一。公鹿今年暑假將面臨陣中自由球員保衛戰，不僅是米道頓，包括米羅蒂奇（Brook Lopez）、波格登（Malcolm Brogdon）、洛培茲（Brook Lopez）等都將尋求新的合約。而對公鹿球團來說，留下這幾位主力不只是維持戰力，同時也是更確保米道頓與安戴托昆波（Giannis Antetokounmpo）的建隊核心主軸。'],
 ['https://nba.udn.com/nba/story/6780/3881035',
  '杜蘭特何去何從？ 匿名教練：在勇士養傷較合適',
  '2019-06-19 16:43',
  '勇士球星杜蘭特（Kevin Durant）季末告別勇士隊傳聞甚囂塵上，但總冠軍賽跟腱斷裂讓他跳脫合約添變數。有匿名NBA主帥建議，與其轉戰受媒體矚目度更高籃網或尼克隊，對養傷的杜蘭特而言，續留勇士是更合適選擇。杜蘭特右小腿拉傷休兵一個月，總冠軍賽復出戰又遭逢右腳阿基里斯腱撕裂，2019-20年球季恐報銷，也為新球季動向留伏筆。消息傳出勇士隊可能開出長約續留，並強調傷勢絕不影響爭取他和湯普森（Klay Thompson）決心。《紐約時報》記者史坦（Marc Stein）表示，就杜蘭特的情況評估，續留加州相對比轉戰紐約更適合，史坦轉述匿名NBA教頭的看法，指

In [10]:

def crawler_main(pages, newest_update):
    main_url = "https://nba.udn.com/nba/index?gr=www"
    domain_url = "https://nba.udn.com"
    # go to main page > 焦點新聞 more
    newest_update = datetime.strptime(newest_update, "%Y-%m-%d %H:%M")
    url = main_url
    response = requests.get(url)
    html = BeautifulSoup(response.text)
    news_page = domain_url + html.find("div", id="mainbar").find("div", id="news").find("h1", class_="box-title").find("a")["href"]
    # https://nba.udn.com/nba/cate/6754
    # result at 0617
    # news_page = "https://nba.udn.com/nba/cate/6754"
    # go page1_newest
    news_articles = []
    for c_page in range(1, pages + 1):
        url = news_page + "/-1/newest/" + str(c_page)
        response = requests.get(url)
        html = BeautifulSoup(response.text)
        body_box = html.find("div", id="news_list_body").find("dl").find_all("dt")
        b = 0
        for box in body_box:
            url = domain_url + box.find("a")["href"]
            update_time = get_article_update_time(url)
            check_update_time = datetime.strptime(update_time, "%Y-%m-%d %H:%M")
            title = box.find("h3").text
            if check_update_time <= newest_update:
                b = 1
                break
            content = get_article_content(url)
            news_articles.append([url, title, update_time, content])
        if b:
            break
    return news_articles


def get_article_update_time(article_url):
    #     article_url = "https://nba.udn.com/nba/story/6780/3877002"
    response = requests.get(article_url)
    html = BeautifulSoup(response.text)
    aritle_update_time = html.find("div", id="shareBar").find("div", class_="shareBar__info") \
        .find("div", class_="shareBar__info--author").find("span").text
    return aritle_update_time


def get_article_content(article_url):
    #     aritle_url = "https://nba.udn.com/nba/story/6780/3877002"
    result = ""
    response = requests.get(article_url)
    html = BeautifulSoup(response.text)
    article_content = html.find("div", id="story_body_content").find_all("p")
    for c in article_content:
        result = result + c.text
    delete_content = " Getty Imagesfacebooktwitterpinterest"
    result = result.replace(delete_content, "")
    return result


# def main():
#     news = crawler_main(2, "2019-06-17 09:54")
#     df = pd.DataFrame(news, columns=["url", "title", "update_time","content"])
#     print(df)
#
#
# main()


In [36]:
url = "https://nba.udn.com/nba/story/6780/3880902"
def get_article_content(article_url):
    #     aritle_url = "https://nba.udn.com/nba/story/6780/3877002"
    result = ""
    response = requests.get(article_url)
    html = BeautifulSoup(response.text)
    article_content = html.find("div", id="story_body_content").find_all("p")
    for c in article_content:
        result = result + c.text
        print(c.text)
        print("==")
        if "pinterest" in c.text :
            result = ""
#     delete_content = " Getty Imagesfacebooktwitterpinterest"
#     result = result.replace(delete_content, "")
#     result = result.split("pinterest")[1:][0]
    return result
res = get_article_content(url)
print(res)

 Getty Images facebooktwitterpinterest
==

==
根據消息，熱火中鋒懷塞德（Hassan Whiteside）計畫將會執行合約中的球員選項，繼續為邁阿密拼戰一年，而他下一季年薪也將達到2710萬美元。
==

==
懷塞德會做如此選擇並不會太讓人感到意外，主要是他上季總共只出賽53場，平均12.3分11.3籃板1.9阻攻，特別是在明星賽過後，懷塞德失去先發地位，熱火改讓阿德巴約（Bam Adebayo）有更多表現空間，這一點曾引發懷塞德的不滿，彼此之間出現不少摩擦。
==

==
在這樣的狀況下，加上現代球風對於這類進攻天賦普通的傳統長人有著明顯的相斥效應，即使懷塞德選擇投入自由市場，很可能無法拿到理想的合約。也因此，懷塞德仍舊選擇走完四年9800萬美元的合約。
==

==
對薪資爆滿的熱火來說，球隊補強最大阻礙就是薪資空間的卡死，陣中擁有多位千萬年薪的球員，球團想要甩賣也面臨買家有限的窘境。
==

==

==
根據消息，熱火中鋒懷塞德（Hassan Whiteside）計畫將會執行合約中的球員選項，繼續為邁阿密拼戰一年，而他下一季年薪也將達到2710萬美元。懷塞德會做如此選擇並不會太讓人感到意外，主要是他上季總共只出賽53場，平均12.3分11.3籃板1.9阻攻，特別是在明星賽過後，懷塞德失去先發地位，熱火改讓阿德巴約（Bam Adebayo）有更多表現空間，這一點曾引發懷塞德的不滿，彼此之間出現不少摩擦。在這樣的狀況下，加上現代球風對於這類進攻天賦普通的傳統長人有著明顯的相斥效應，即使懷塞德選擇投入自由市場，很可能無法拿到理想的合約。也因此，懷塞德仍舊選擇走完四年9800萬美元的合約。對薪資爆滿的熱火來說，球隊補強最大阻礙就是薪資空間的卡死，陣中擁有多位千萬年薪的球員，球團想要甩賣也面臨買家有限的窘境。


In [24]:
str1 = "12345aa5agaabbb"
"".join(str1.split("5")[1:])

'aaagaabbb'

In [5]:
from crawler_NBA import crawler_main
import warnings
from datetime import datetime
import sqlite3
warnings.filterwarnings("ignore")
# 先檢查有最新資料
conn = sqlite3.connect("../db.sqlite3")
c = conn.cursor()
sel = c.execute("select update_time from news order by update_time -1 limit 1")
conn.commit()
lastest_update = ""
for data in sel:
        lastest_update = data[0]
        print(lastest_update)
if lastest_update == "":
    lastest_update = "2019-06-15 00:00"
conn.close()

# 依照最新資料去爬
# default 爬最新的五頁 ~ 最新資料為止

news = crawler_main(2, lastest_update)
insert_data = []
print("total:", len(news), "data get")
for data in news:
    url, title, update_time, content = data
#     update_time = datetime.strptime(update_time, "%Y-%m-%d %H:%M")
    created = datetime.now().strftime("%Y-%m-%d %H:%M")
    insert_data.append((url, title, update_time, content,created,))

conn = sqlite3.connect("../db.sqlite3")
c = conn.cursor()
c.executemany('INSERT INTO news VALUES (null,?,?,?,?,?)', insert_data)
conn.commit()
conn.close()
print("data_updated")

total: 20 data get
data_updated


In [1]:
# # TRUNCATE TABLE
# from crawler_NBA import crawler_main
# import warnings
# from datetime import datetime
# import sqlite3
# warnings.filterwarnings("ignore")
# # 先檢查有最新資料
# conn = sqlite3.connect("../db_backup.sqlite3")
# c = conn.cursor()
# c.execute("DELETE FROM news")
# conn.commit()
# # for data in sel:
# #     lastest = data[0]
# conn.close()
# # print(lastest)